# Introduction to yabadaba: All about Query objects

This Notebook provides a quick overview on Query objects.  

Most users will likely not need to mess around here either because a) the built-in Value classes already have default Queries associated with them, and b) custom Value classes tend to be for more complex data that is often not meaningful to query against.  This is mostly for users who have defined their own Value classes that contains data that can be useful and practical to query against.

## 1. What are Query objects?

The Query objects manage how user-friendly database queries are built in yabadaba.  The idea is

1. When a user performs a database query (i.e. calling Database.get_records()), the querying method accepts keyword arguments and simple values that specify how to limit the set of returned records.
2. Each of those recognized keyword arguments is associated with a Query object with a style that indicates the type of query operation that should be performed.
3. In turn, each Query object is associated with a Value object indicating which value in the record the query operation checks.
4. The methods of a Query class detail how to perform the query operation in an efficient way to different database infrastructures.
5. As the different Database class styles are tied to specific database infrastructures, each Database subclass can access the appropriate version of the Query operation methods.
6. Thus, regardless of the database infrastructure, simple query operations can be performed in efficient and universal ways.

## 2. Built-in Query styles

The yabadaba package defines a number of Query classes, most of which are associated with the simple data types of the built-in Value objects. 

- __'bool_match'__ filters based on if a boolean element matches a given value.
- __'str_contains'__ filters based on if a str element contains at least one given value.
- __'str_match'__ filters based on if a str element matches at least one given value.
- __'list_contains'__ filters based on if a list/array element contains at least one given value.
- __'int_match'__ filters based on if an int element matches at least one given value.
- __'float_match'__ filters based on if a float element matches at least one given value.
- __'date_match'__ filters based on if a datetime.date element matches at least one given value.
- __'month_match'__ filters based on if a month element matches at least one given value.
- __'dummy'__ doesn't do any filtering based on record contents for the associated query term. It does, however, allow for optional filtering based on if one of the given values matches some pre-defined constant value, i.e. return all if the correct value is wanted and return none otherwise.  This behavior can be useful if you want multiple Record classes to have the same set of query terms even if some of the terms do not apply to all of the classes.

## 3. Specifying Queries

### 3.1. Default queries for Value classes

The typical way of specifying what queries to include is to do so by including them in a Value class's _default_queries dict. The _default_queries property method should

1. Return an empty dict if self.metadatakey is False.  This allows for queries to be turned off if the value is excluded from the metadata record representation.
2. Otherwise, build and return a dict where the keys correspond to the keyword arguments of get_record(s) to associate with the query and the values are Query objects.
3. Typically, the dict keys should be the Value object's name (self.name) or should include it to indicate which value is being queried against.
4. The value objects can be generated by calling yabadaba.load_query().

The parameters for yabadaba.load_query are

- __style__ (*str*) The query style.
- __name__ (*str or None, optional*) The metadata key associated with the data field.  Must be set to use the pandas query method.
- __parent__ (*str or None, optional*) Allows for the pandas query operations to work on embedded metadata dicts.  If given, the pandas query method will check the value of metadata\[parent\]\[name\].
- __path__ (*str or None, optional*) The record data path to the data field.  Levels are delimited by periods.  Must be given to use the mongo query method.
- __description__ (*str, optional*) Description of the query operation, i.e. what it is searching.  Typically, this is a f-string allowing for the value's name to be included.
- __**kwargs__ (*any, optional*) Any additional style-specific keyword parameters.
 
While most of the built-in Value classes contain a single Query, there is no limit on the number of Queries that can be associated with a Value.  Some examples where this could be used are

- You have a value that can be represented by a simple data type, but you wish for multiple query operations that can operate on that same value.  Some examples are a string-based value that you want both str_match and str_contains options, or a numerical value that you want to query by ranges using separate custom float_min and float_max queries.
- You have a value that represents some complex data that perhaps is interpreted as a Python object when loading. The object itself is not easily queryable, but the data model representation contains sub-fields that are.  In this case, multiple queries can be defined for different sub-fields.  In this case, you will likely need to modify the Value's metadata() method so that the multiple queryable sub-fields appear in it as well.

### 3.2. Defining new Query classes

New Query classes can be defined in fashion similar to how new Value and Record classes are defined. The only difference being that the Query subclasses inherit from yabadaba.query.Query, and are integrated into yabadaba using yabadaba.querymanager's import_style() method.

The methods and attributes of Query to override in the subclasses are

- __style__ specifies the Query style and should match the style assigned to the Query during the querymanager.import_style() call.
- __parameter_type__ is a str listing the allowed data types that the corresponding query parameter for the Database.get_record(s) call can be.  This is only used to generate the query description when Record.querydoc is called.
- __mongo()__ builds the necessary Mongo-style query dict(s) for the query operation, which is then appended to the list of all other Mongo-stype query dicts generated by the other Query objects of a Record.  Defining this requires knowledge of how to build Mongo-style queries. This is used by both the MongoDatabase and CDCSDatabase classes.
- __pandas()__ takes a pandas.DataFrame containing the metadata content for multiple Record objects of the same style and returns a bool mask indicating which rows agree/disagree with the query operation.  This is typically done by defining a row-based evaluation function and applying it to the DataFrame.  Defining this requires knowledge of how to interact with data in a pandas.DataFrame. This is used by the LocalDatabase class.

Similar to Value classes, you can also override other methods of the base Query class as needed.  The Query class is a bit simpler, so the main other method to possibly override is the \_\_init\_\_() if additional parameter settings are needed when creating Query objects. 
